In [17]:
!pip install -r ./requirements.txt -q

In [18]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline import Pipeline

from sagemaker.workflow.function_step import step
from sagemaker import get_execution_role, Session
import logging

In [19]:
from steps.fetch import fetch
from steps.preprocess import preprocess
from steps.train import train
from steps.eval import eval

In [20]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

In [21]:
sagemaker_session = Session()
role = get_execution_role()

default_bucket = sagemaker_session.default_bucket()
session = PipelineSession(boto_session=sagemaker_session.boto_session, default_bucket=default_bucket)
pipeline_name = 'stock-pipeline'
default_ticker = "SPY"
years_of_data_to_fetch = 1

In [22]:
# @step(
#     instance_type="ml.m5.large"
# )
# def train(data):
#     # retrieve preprocessed csv data
#     # train model
#     # save model to s3 bucket /model with postfix of date and RMSE in filename
#     logging.info(f'data should be output of preprocess function: {data}')
#     return "stub_train"

In [23]:
# @step(
#     instance_type="ml.m5.large"
# )
# def eval(data):
#     # load current model file and latest
#     # extract model performance from RSME in filename
#     # if model is better than current
#     # pass True to deploy phase (Use conditional to assess when to run deploy)
#     logging.info(f'data should be output of preprocess function: {data}')
#     return False

In [24]:
# @step(
#     instance_type="ml.m5.large"
# )
def deploy(data):
    # if latest_model is different than current
    # update deployment to use latest model
    logging.info(f'data should be output of train function: {data}')
    return "stub_eval"

In [25]:
fetch_result = fetch(default_ticker, years_of_data_to_fetch)
print(fetch_result)

INFO:steps.fetch:Successfully fetched data for SPY
INFO:steps.fetch:Raw data saved to s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv


s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv


In [26]:
preprocess_result = preprocess(fetch_result)
print(preprocess_result)

INFO:steps.preprocess:Starting preprocessing with data from s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv
INFO:steps.preprocess:Successfully read raw data from s3://sagemaker-us-east-1-971173012767/raw/SPY_raw_data.csv
/home/sagemaker-user/aai-540_stock_trading_ai/cloud/steps/preprocess.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data.sort_values('Date', inplace=True)
INFO:steps.preprocess:Data preprocessing completed
INFO:steps.preprocess:Preprocessed data saved to s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv


s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv


In [27]:
train_result = train(preprocess_result)
print(train_result)

INFO:steps.train:Starting training with data from s3://sagemaker-us-east-1-971173012767/processed/processed/SPY_processed_data.csv
INFO:steps.train:Loaded preprocessed data with shape (250, 5)
ERROR:steps.train:Error during training: could not convert string to float: '2024-02-23'


ValueError: could not convert string to float: '2024-02-23'

In [ ]:
eval_result = eval(train_result)
print(eval_result)

In [ ]:
deploy_result = deploy(eval_result)
print(deploy_result)

In [7]:

deploy_result = deploy(eval_result)

pipeline = Pipeline(
    name=pipeline_name,
    steps=[fetch_result, preprocess_result, train_result, eval_result, deploy_result],
    sagemaker_session=session,
)

In [8]:
pipeline.upsert(role)

2025-02-21 20:50:38,084 sagemaker.remote_function INFO     Uploading serialized function code to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-80b1c5ef-0734-4a54-9399-a435f0087fff/2025-02-21-20-50-37-995/function
2025-02-21 20:50:38,258 sagemaker.remote_function INFO     Uploading serialized function arguments to s3://sagemaker-us-east-1-971173012767/stock-pipeline/fetch-80b1c5ef-0734-4a54-9399-a435f0087fff/2025-02-21-20-50-37-995/arguments


ValueError: Invalid dependencies provided: "cloud/"

In [ ]:
pipeline.start()